In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import umap
import torch
import torch.nn as nn
import torch.nn.functional as F
import gc
from tqdm import tqdm

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.chdir('../IConNet/')
os.getcwd()

'/home/linh/projects/IConNet'

In [5]:
from einops import rearrange, reduce, repeat

In [6]:
dataset_name = 'meld'
experiment_prefix = "scb13"
log_dir = f'../{experiment_prefix}_models/meld/'
codebook_pretrained_path = f'../scb11_models/ravdess/epoch=220.masked_codebook.pt'
data_dir = "../data/data_preprocessed/"
sr = 16000


In [7]:
data_x = np.load(f'{data_dir}{dataset_name}/preprocessing/{dataset_name}.audio16k.npy', allow_pickle=True)
print(data_x.shape)

(4639,)


In [8]:
data_y = np.load(f'{data_dir}{dataset_name}/preprocessing/{dataset_name}.label_emotion_key.npy', allow_pickle=True)
print(data_y.shape)
np.unique(data_y)

(4639,)


array(['ang', 'hap', 'neu', 'sad'], dtype=object)

In [9]:
import pandas as pd
from IConNet.trainer.train_torch import get_dataloader
from IConNet.trainer.train_torch import Trainer_custom_model as Trainer
from IConNet.acov.model import SCB13 as SCB
from omegaconf import OmegaConf as ocf

sr = 16000
batch_size = 2
in_channels = 1
kernel_size = 511
stride = 125
embedding_dim = 511
num_embeddings = 384
cls_dim = 512
learning_rate = 1e-4
num_classes = 4
window_k = 5

model = SCB(
    in_channels=in_channels,    
    num_embeddings=num_embeddings, 
    stride=1,
    window_k=window_k,
    embedding_dim=embedding_dim, 
    num_classes=num_classes, 
    cls_dim=cls_dim,
    sample_rate=sr,
    codebook_pretrained_path=codebook_pretrained_path,
)

In [10]:
import IPython.display as ipd
import seaborn as sns

from IConNet.acov.visualize import (
    visualize_speech_codebook, get_embedding_color, 
    visualize_embedding_umap, visualize_training_curves,
    get_embedding_color_v2, get_zcs_color_v2
)

In [11]:
dataset_config_path = f'config/dataset/{dataset_name}.yaml'
dataset_config = ocf.load(dataset_config_path)
print(dataset_config)

{'name': 'meld', 'dataset_class': 'WaveformDataset', 'root': 'meld/', 'audio_dir': 'full_release/', 'feature_dir': 'preprocessing/', 'label_name': 'label_emotion_key', 'feature_name': 'audio16k', 'num_classes': 4, 'label_values': ['neu', 'hap', 'sad', 'ang'], 'classnames': ['neu', 'hap', 'sad', 'ang'], 'target_labels': ['ang', 'neu', 'sad', 'hap']}


In [12]:
train_loader, test_loader, batch_size = get_dataloader(
    dataset_config, data_dir, batch_size=2)
print(batch_size)

2


In [13]:
trainer = Trainer(batch_size=batch_size, log_dir=log_dir, 
                  experiment_prefix=experiment_prefix, device=device,
                 accumulate_grad_batches=16, gradient_clip_val=1.)

trainer.prepare(train_loader=train_loader, 
                test_loader=test_loader, 
                batch_size=batch_size)
trainer.setup(model, lr=1e-4)

In [14]:
trainer.fit(n_epoch=100, test_n_epoch=10)

  0%|▏                          | 0.4766307139188397/100 [02:43<385:32:25, 13945.93s/it]

Epoch: 1	Loss: 1.295	Val_acc: 45/185 (24.32%)

Saved new best val model: ../scb13_models/meld/model.epoch=1.step=870.loss=1.295.val_acc=0.243.pt


  1%|▎                          | 0.9527478171545639/100 [05:26<382:30:05, 13902.52s/it]

Epoch: 1	Loss: 1.910	Val_acc: 51/185 (27.57%)

Saved new best val model: ../scb13_models/meld/model.epoch=1.step=1739.loss=1.910.val_acc=0.276.pt


  1%|▍                              | 1.429378531073483/100 [09:14<13:07:20, 479.25s/it]

...

Epoch: 99	Loss: 0.522	Val_acc: 49/185 (26.49%)



100%|████████████████████████████▊| 99.5649717512084/100 [9:51:53<1:42:56, 14196.81s/it]

Epoch: 100	Loss: 1.510	Val_acc: 51/185 (27.57%)



100%|███████████████████████████████▉| 99.99999999979435/100 [9:54:03<00:00, 307.20s/it]/opt/anaconda3/envs/audio/lib/python3.9/site-packages/tqdm/std.py:639: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|████████████████████████████| 100.04108885444238/100 [9:54:40<-1:59:46, 356.66s/it]

Epoch: 100	Loss: 1.180	Val_acc: 53/185 (28.65%)



In [15]:
trainer.fit(n_epoch=100, test_n_epoch=10)

  0%|▏                          | 0.4766307139188397/100 [02:47<393:47:18, 14244.28s/it]

Epoch: 101	Loss: 1.486	Val_acc: 48/185 (25.95%)



  1%|▎                              | 0.9522342064714617/100 [05:10<8:21:50, 304.00s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7b5c2f971070>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/audio/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
  1%|▎                              | 0.9522342064714617/100 [05:16<9:08:42, 332.39s/it]

KeyboardInterrupt



In [16]:
codebook_pretrained_path = f'../scb11_models/ravdess/epoch=220.codebook.pt'

In [21]:
window_k = 9

model = SCB(
    in_channels=in_channels,    
    num_embeddings=num_embeddings, 
    stride=1,
    window_k=window_k,
    embedding_dim=embedding_dim, 
    num_classes=num_classes, 
    cls_dim=cls_dim,
    sample_rate=sr,
    codebook_pretrained_path=codebook_pretrained_path,
)

In [22]:
trainer = Trainer(batch_size=batch_size, log_dir=log_dir, 
                  experiment_prefix=experiment_prefix, device=device,
                 accumulate_grad_batches=16, gradient_clip_val=1.)

trainer.prepare(train_loader=train_loader, 
                test_loader=test_loader, 
                batch_size=batch_size)
trainer.setup(model, lr=1e-4)

In [23]:
trainer.fit(n_epoch=100, test_n_epoch=10)

  0%|▏                          | 0.4766307139188397/100 [02:54<416:12:06, 15055.02s/it]

Epoch: 1	Loss: 1.581	Val_acc: 53/185 (28.65%)

Saved new best val model: ../scb13_models/meld/model.epoch=1.step=870.loss=1.581.val_acc=0.286.pt


  1%|▎                          | 0.9527478171545639/100 [05:51<420:14:03, 15273.95s/it]

Epoch: 1	Loss: 1.709	Val_acc: 51/185 (27.57%)



  1%|▍                              | 1.429378531073483/100 [09:59<14:04:15, 513.91s/it]

...

 11%|███▍                          | 11.4334874165359/100 [1:12:45<12:47:47, 520.14s/it]

Correct: 260/928 (0.2802)
{'acc_unweighted': tensor(0.2520, device='cuda:0'),
 'acc_weighted': tensor(0.2802, device='cuda:0'),
 'f1s_unweighted': tensor(0.1146, device='cuda:0'),
 'f1s_weighted': tensor(0.1274, device='cuda:0'),
 'rocauc': tensor(0.4988, device='cuda:0'),
 'uar': tensor(0.2520, device='cuda:0'),
 'wap': tensor(0.4477, device='cuda:0')}
{'acc_detail': tensor([0.9961, 0.0040, 0.0000, 0.0077], device='cuda:0'),
 'f1s_detail': tensor([0.4352, 0.0080, 0.0000, 0.0152], device='cuda:0'),
 'precision_detail': tensor([0.2784, 0.3333, 0.0000, 1.0000], device='cuda:0'),
 'recall_detail': tensor([0.9961, 0.0040, 0.0000, 0.0077], device='cuda:0'),
 'rocauc_detail': tensor([0.4717, 0.5140, 0.4913, 0.5180], device='cuda:0')}
tensor([[257,   1,   0,   0],
        [247,   1,   0,   0],
        [160,   1,   0,   0],
        [259,   0,   0,   2]], device='cuda:0')


 12%|██▉                      | 11.910118130454624/100 [1:15:42<369:00:48, 15080.60s/it]

Epoch: 12	Loss: 1.413	Val_acc: 41/185 (22.16%)



 12%|███▍                         | 11.929635336412503/100 [1:15:48<7:39:56, 313.35s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7b5c2f971070>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/audio/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
 12%|███▍                         | 11.930148947095605/100 [1:15:48<9:19:40, 381.29s/it]

KeyboardInterrupt

